In [46]:
import requests 
import json  
from bs4 import BeautifulSoup
import json
import re
import pyodbc
import math 

In [ ]:
   

# --- 1. Configuration (les paramètres de ton pipeline) ---
SUBREDDIT = "headphones"
MOT_CLE = "Sony XM5" # Un mot-clé de ton fichier config

# --- 2. Construction de l'URL et des Paramètres ---
# On utilise l'endpoint de RECHERCHE de Reddit, au format JSON
url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"

# Paramètres de la recherche :
params = {
    'q': MOT_CLE,        # 'q' = query (le mot-clé que tu cherches)
    'sort': 'new',       # 'new' = trier par "plus récent" (parfait pour ton pipeline)
    'restrict_sr': 'true', # 'true' = restreindre la recherche à ce subreddit
    'limit': 10          # On veut 10 résultats
}

# !! TRÈS IMPORTANT !!
# Reddit bloque les scripts qui n'ont pas de "User-Agent".
# On doit simuler un navigateur pour être poli et éviter un blocage.
headers = {
    'User-Agent': 'MonProjetDataEngineering-v0.1'
}

# --- 3. Exécution de l'Appel API ---
print(f"Appel de l'API Reddit pour '{MOT_CLE}' dans r/{SUBREDDIT}...")

try:
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status() # Vérifie s'il y a eu une erreur (ex: 404, 500)

    # --- 4. Affichage du Résultat ---
    data = response.json() # Convertit la réponse texte en objet JSON

    # Les posts sont imbriqués dans cette structure
    posts = data['data']['children']

    if not posts:
        print(f"\n Pas de nouveaux posts trouvés pour '{MOT_CLE}'. ---")
    else:
        print(f"\n{len(posts)} posts trouvés : ---")
        
        # --- 5. Début de la Transformation (ce que fera ton ETL) ---
        for i, post in enumerate(posts):
            post_data = post['data']
            
            titre = post_data['title']
            texte_brut = post_data['selftext'] # Le corps du post
            
            print(f"\n--- Post {i+1} (ID: {post_data['id']}) ---")
            print(f"Titre: {titre}")
            
            
            # Ton script d'analyse de sentiment lira ce titre et ce texte
            # ... (Étape suivante: appliquer VADER ou TextBlob ici) ...


except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
except requests.exceptions.RequestException as e:
    print(f"\n--- ❌ ERREUR de Connexion : {e} ---")
except KeyError:
    print("\n--- ❌ ERREUR de Parsing JSON ---")
    print("La structure de la réponse de Reddit a peut-être changé.")
    print("Réponse brute reçue :", response.text[:200] + "...")

Appel de l'API Reddit pour 'Sony XM5' dans r/headphones...

--- ✅ SUCCÈS ! 10 posts trouvés : ---

--- Post 1 (ID: 1odad1t) ---
Titre: What are these?

--- Post 2 (ID: 1occfmu) ---
Titre: Audiophile Verdict of Bose QC Ultra 2 Headphones: DO NOT BUY 1ST GEN!

--- Post 3 (ID: 1o1oqjf) ---
Titre: Motion sickness from Sennheiser M4, should i switch to other brands?

--- Post 4 (ID: 1nuylfe) ---
Titre: Just bought Arya Stealth coming from Sony XM5 over ears. Will I notice a difference?

--- Post 5 (ID: 1nf3m4a) ---
Titre: Why do I struggle to enjoy over-ear headphones like the Sony XM4 or XM5, and how can I get used to them and start liking the experience?

--- Post 6 (ID: 1n16nhe) ---
Titre: How to disinfect/clean ear cups?

--- Post 7 (ID: 1n13i3p) ---
Titre: Bose QCU #FAIL Sony #FAIL are there any decent options left?

--- Post 8 (ID: 1myfywb) ---
Titre: Sonos Ace!

--- Post 9 (ID: 1mjxnrh) ---
Titre: Sony xm5 pros and cons

--- Post 10 (ID: 1m27bsa) ---
Titre: XM5


In [ ]:

# --- 1. Configuration ---
URL_DECOUVERTE = "https://www.fnac.com/Casque-Bluetooth-sans-fil/Casque-par-usage/nsh450503/w-4?SDM=list&ssi=6&sso=2"

# --- NOUVEAU : En-têtes plus complets ---
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7', # Préférer le français
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.google.com/', # Simule une venue depuis Google
    'DNT': '1', # Do Not Track
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# --- 2. Scraping de la page ---
print(f"Appel de l'URL : {URL_DECOUVERTE}...")
try:
    # Utilisation d'une session pour potentiellement gérer les cookies si nécessaire
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_DECOUVERTE)
    response.raise_for_status() # Lève une exception pour les codes 4xx/5xx

    # ... (le reste de ton code pour parser avec BeautifulSoup reste le même) ...
    # ... (trouver le div#FnacContent, extraire data-state, parser le JSON...) ...
    
    soup = BeautifulSoup(response.content, 'html.parser')
    main_div = soup.find('div', id='FnacContent')
    
    if not main_div or 'data-state' not in main_div.attrs:
        print("\n--- ❌ ERREUR : Impossible de trouver le JSON 'data-state'. ---")
    else:
        json_string = main_div['data-state']
        data = json.loads(json_string)
        references = data.get('references', [])
        
        if not references:
            print("\n--- ⚠️ Pas de produits trouvés dans le JSON 'references'. ---")
        else:
            print(f"\n--- ✅ SUCCÈS : {len(references)} produits découverts via JSON ---")
            liste_produits = []
            for ref in references:
                prid = ref.get('prid')
                if prid:
                    # !! VÉRIFIE CE FORMAT D'URL !!
                    product_url = f"https://www.fnac.com/a{prid}/w-4" 
                    liste_produits.append({"prid": prid, "url": product_url})

            print("\n--- Liste des produits découverts (PRID et URL) : ---")
            for p in liste_produits[:5]: 
                 print(p)


except requests.exceptions.HTTPError as err:
    # Affichage plus détaillé de l'erreur 403
    print(f"\n--- ❌ ERREUR HTTP : {err} ---") 
    if response.status_code == 403:
        print("   Cause probable : Blocage anti-scraping par le serveur.")
        print("   Vérifie les 'headers'. Si ça persiste, une solution de proxy pourrait être nécessaire.")
    else:
        print("   Le serveur a retourné une erreur.")
except json.JSONDecodeError:
    print("\n--- ❌ ERREUR : Impossible de parser le JSON dans 'data-state'. ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Appel de l'URL : https://www.fnac.com/Casque-Bluetooth-sans-fil/Casque-par-usage/nsh450503/w-4?SDM=list&ssi=6&sso=2...

--- ❌ ERREUR HTTP : 403 Client Error: Forbidden for url: https://www.fnac.com/Casque-Bluetooth-sans-fil/Casque-par-usage/nsh450503/w-4?SDM=list&ssi=6&sso=2 ---
   Cause probable : Blocage anti-scraping par le serveur.
   Vérifie les 'headers'. Si ça persiste, une solution de proxy pourrait être nécessaire.


In [ ]:


# --- 1. Configuration ---
URL_TEST = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"

# En-têtes pour simuler un navigateur
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.google.com/',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

# --- 2. Exécution du Test ---
print(f"Tentative d'accès à : {URL_TEST}...")
try:
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_TEST, timeout=10) # Ajout d'un timeout
    response.raise_for_status() # Lève une exception pour les codes 4xx/5xx

    # --- 3. Résultat ---
    print(f"\n--- ✅ SUCCÈS ! Code statut : {response.status_code} ---")
    print("Le site semble accessible au scraping basique.")
    # On pourrait ajouter ici une vérification rapide du contenu pour être sûr
    # print(f"Contenu reçu (premiers 200 chars): {response.text[:200]}...")

except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
    if response.status_code == 403:
        print("   Cause probable : Blocage anti-scraping (similaire à FNAC).")
    else:
        print(f"   Le serveur a retourné une erreur {response.status_code}.")
except requests.exceptions.Timeout:
     print("\n--- ❌ ERREUR : La requête a expiré (Timeout). Le serveur est peut-être lent ou bloque.")
except requests.exceptions.RequestException as e:
    print(f"\n--- ❌ ERREUR de Connexion : {e} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Tentative d'accès à : https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque...

--- ✅ SUCCÈS ! Code statut : 200 ---
Le site semble accessible au scraping basique.


In [ ]:

URL_DECOUVERTE = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"
BASE_URL = "https://www.vandenborre.be"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.google.com/',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

print(f"Scraping : {URL_DECOUVERTE}...")
try:
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_DECOUVERTE, timeout=15)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    product_containers = soup.find_all('div', {'class': 'js-product-container'})

    if not product_containers:
        print("\n--- ⚠️ Aucun conteneur produit trouvé avec 'div.js-product-container'. Vérifie les sélecteurs. ---")

    print(f"\n--- ✅ {len(product_containers)} conteneurs produits trouvés ---")

    produits_decouverts = []

    for container in product_containers:
        product_id = container.get('data-productid')
        if not product_id:
            continue

        product_data = {"product_id": product_id}

        # URL et Nom
        name_tag = container.find('h2', {'class': 'productname'})
        link_tag = container.find('a', {'class': 'js-product-click'})
        if name_tag and link_tag and link_tag.get('href'):
            product_data["name"] = name_tag.text.strip()
            # Construit l'URL complète
            relative_url = link_tag['href']
            if relative_url.startswith('//'):
                product_data["url"] = f"https:{relative_url}"
            elif relative_url.startswith('/'):
                 product_data["url"] = f"{BASE_URL}{relative_url}"
            else:
                 product_data["url"] = relative_url # Au cas où elle serait déjà complète
        else:
            product_data["name"] = "Nom non trouvé"
            product_data["url"] = "URL non trouvée"

        # Prix
        price_tag = container.find('span', {'class': 'current'})
        if price_tag:
            price_text = price_tag.text.strip().replace('€', '').replace(',', '.').replace('\xa0', '').replace(' ', '')
            try:
                product_data["price"] = float(re.sub(r'[^\d\.]', '', price_text))
            except (ValueError, TypeError):
                 product_data["price"] = None
        else:
            product_data["price"] = None

        # Note et Avis
        rating_score_tag = container.find('div', {'class': 'rating-score'})
        review_count_tag = container.find('div', {'class': 'rating-reviews-amount'})

        if rating_score_tag and rating_score_tag.find('strong'):
            rating_text = rating_score_tag.find('strong').text.strip().replace(',', '.')
            try:
                product_data["rating"] = float(rating_text)
            except (ValueError, TypeError):
                 product_data["rating"] = None
        else:
             product_data["rating"] = None

        if review_count_tag and review_count_tag.find('a'):
            review_text = review_count_tag.find('a').text.strip()
            count_match = re.search(r'\((\d+)\)', review_text)
            if count_match:
                 try:
                    product_data["review_count"] = int(count_match.group(1))
                 except (ValueError, TypeError):
                    product_data["review_count"] = None
            else:
                 product_data["review_count"] = None
        else:
            product_data["review_count"] = None
            
        # Marque (simple extraction du premier mot du nom)
        if product_data["name"] != "Nom non trouvé":
             product_data["brand"] = product_data["name"].split(' ')[0]
        else:
             product_data["brand"] = None


        produits_decouverts.append(product_data)

    print("\n--- Données extraites (5 premiers produits) : ---")
    if produits_decouverts:
        print(json.dumps(produits_decouverts[:5], indent=2, ensure_ascii=False))
    else:
        print("Aucun produit n'a pu être extrait.")


except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
    if response and response.status_code == 403:
        print("   Cause : Blocage anti-scraping.")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Scraping : https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque...

--- ✅ 29 conteneurs produits trouvés ---

--- Données extraites (5 premiers produits) : ---
[
  {
    "product_id": "7819145",
    "name": "JBL TUNE 770NC BLACK",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black",
    "price": 89.0,
    "rating": 4.4,
    "review_count": 70,
    "brand": "JBL"
  },
  {
    "product_id": "7683081",
    "name": "SONY WH-1000XM6 NOIR",
    "url": "https://www.vandenborre.be/fr/casque/sony-wh-1000xm6-noir",
    "price": 449.0,
    "rating": 4.8,
    "review_count": 17,
    "brand": "SONY"
  },
  {
    "product_id": "7762429",
    "name": "JBL LIVE 770NC NOIR",
    "url": "https://www.vandenborre.be/fr/casque/jbl-live-770nc-noir",
    "price": 120.0,
    "rating": 4.5,
    "review_count": 147,
    "brand": "JBL"
  },
  {
    "product_id": "7825773",
    "name": "JBL TUNE 520 BT BLACK",
    "url": "https://www.vandenborre.be/fr/casque/jbl-tune-520-bt-black",
    "pr

In [ ]:


# --- 1. Configuration ---
SUBREDDIT = "headphones"
# Liste des mots-clés correspondant aux produits trouvés
PRODUITS_A_TESTER = [
    "JBL Tune 770NC", 
    "Sony WH-1000XM6", 
    "JBL Live 770NC" 
]

headers = {
    'User-Agent': 'MonProjetDataEngineering-TestMention-v0.1' 
}

# --- 2. Boucle de Test ---
print(f"Test de mentions dans r/{SUBREDDIT}...\n")
produits_avec_mentions = 0

for mot_cle in PRODUITS_A_TESTER:
    print(f"--- Recherche de '{mot_cle}' ---")
    
    url = f"https://www.reddit.com/r/{SUBREDDIT}/search.json"
    params = {
        'q': mot_cle,
        'sort': 'new',
        'restrict_sr': 'true',
        'limit': 5 # On ne cherche que 5 posts pour ce test
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        data = response.json()
        posts = data['data']['children']

        if not posts:
            print(f"   -> ⚠️ Aucune mention récente trouvée pour '{mot_cle}'.")
        else:
            print(f"   -> ✅ Trouvé {len(posts)} mentions récentes pour '{mot_cle}'.")
            # Afficher le titre du premier post trouvé pour vérification
            print(f"      Exemple: '{posts[0]['data']['title'][:80]}...'")
            produits_avec_mentions += 1
            
    except requests.exceptions.HTTPError as err:
        print(f"   -> ❌ ERREUR HTTP lors de la recherche de '{mot_cle}': {err}")
    except Exception as e:
        print(f"   -> ❌ ERREUR INATTENDUE lors de la recherche de '{mot_cle}': {e}")
        
    # Pause de politesse pour respecter les limites de Reddit
    time.sleep(1) 

# --- 3. Conclusion du Test ---
print("\n--- Résultat du Test ---")
if produits_avec_mentions > 0:
    print(f"✅ Confirmation : Au moins {produits_avec_mentions}/{len(PRODUITS_A_TESTER)} produits testés ont des mentions récentes sur Reddit.")
    print("   -> La Source 3 (Sentiment) semble viable.")
else:
     print("❌ Problème : Aucun des produits testés n'a de mention récente sur Reddit.")
     print("   -> La Source 3 (Sentiment Reddit) pourrait être difficile à alimenter pour ce marché.")
     print("   -> Envisage d'élargir les mots-clés ou de changer de subreddit.")

Test de mentions dans r/headphones...

--- Recherche de 'JBL Tune 770NC' ---
   -> ✅ Trouvé 5 mentions récentes pour 'JBL Tune 770NC'.
      Exemple: 'I have no idea which headphone to go ahead with after this one broke...'
--- Recherche de 'Sony WH-1000XM6' ---
   -> ✅ Trouvé 5 mentions récentes pour 'Sony WH-1000XM6'.
      Exemple: 'first impression on sony’s WH-1000XM6, i have one complaint...'
--- Recherche de 'JBL Live 770NC' ---
   -> ✅ Trouvé 5 mentions récentes pour 'JBL Live 770NC'.
      Exemple: 'I saw someone wearing the xm4’s...'

--- Résultat du Test ---
✅ Confirmation : Au moins 3/3 produits testés ont des mentions récentes sur Reddit.
   -> La Source 3 (Sentiment) semble viable.


In [10]:
import requests
from bs4 import BeautifulSoup
import json
import re

URL_PRODUIT = "https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black"
BASE_URL = "https://www.vandenborre.be"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}

print(f"Scraping (JSON Caché) : {URL_PRODUIT}...")
try:
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_PRODUIT, timeout=15)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, 'html.parser')

    # --- 3. Extraction du JSON (Version corrigée) ---
    
    # Trouve TOUS les scripts JSON-LD
    json_scripts = soup.find_all('script', {'type': 'application/ld+json'})
    
    product_data = None # Variable pour stocker le bon JSON

    for script in json_scripts:
        if not script.string:
            continue
            
        try:
            data = json.loads(script.string)
            
            # Cas 1: Le JSON est un dictionnaire
            if isinstance(data, dict) and data.get("@type") == "Product":
                product_data = data
                break # On a trouvé le bon JSON, on arrête la boucle
                
            # Cas 2: Le JSON est une liste de dictionnaires
            if isinstance(data, list):
                for item in data:
                    if isinstance(item, dict) and item.get("@type") == "Product":
                        product_data = item
                        break # On a trouvé le bon JSON
            if product_data:
                break

        except json.JSONDecodeError:
            continue # Ignorer les scripts JSON mal formés

    # --- 4. Affichage du Résultat ---
    if not product_data:
        print("\n--- ❌ ERREUR : Impossible de trouver le JSON '@type': 'Product' dans la page. ---")
    else:
        print("\n--- ✅ SUCCÈS ! Données JSON 'Product' extraites : ---")
        
        # Données pour Dim_Product
        print("\n--- Pour Dim_Product (Catalogue) ---")
        print(f"SKU: {product_data.get('sku')}")
        print(f"Nom: {product_data.get('name')}")
        print(f"Marque: {product_data.get('brand', {}).get('name')}")
        print(f"Catégorie: {product_data.get('category')}")
        
        # Données pour Fact_Marketplace_Snapshot
        print("\n--- Pour Fact_Marketplace_Snapshot (Performances) ---")
        print(f"Prix: {product_data.get('offers', {}).get('price')}")
        print(f"Note: {product_data.get('aggregateRating', {}).get('ratingValue')}")
        print(f"Nb Avis: {product_data.get('aggregateRating', {}).get('reviewCount')}")
        print(f"Dispo: {product_data.get('offers', {}).get('availability')}")


except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")

Scraping (JSON Caché) : https://www.vandenborre.be/fr/casque/jbl-tune-770nc-black...

--- ✅ SUCCÈS ! Données JSON 'Product' extraites : ---

--- Pour Dim_Product (Catalogue) ---
SKU: 7819145
Nom: JBL TUNE 770NC BLACK
Marque: JBL
Catégorie: Casque audio

--- Pour Fact_Marketplace_Snapshot (Performances) ---
Prix: 89
Note: 4.4
Nb Avis: 70
Dispo: https://schema.org/InStock


In [ ]:

DB_CONFIG = {
    'server': 'LAPTOP-VT8FTHG2\DATAENGINEER', # ex: '.\SQLEXPRESS' ou 'MON-PC\NOM_INSTANCE'
    'database': 'Projet_Market_Staging',
    'driver': '{ODBC Driver 17 for SQL Server}',
    'connection_string': (
        "DRIVER={ODBC Driver 17 for SQL Server};"
        "SERVER=LAPTOP-VT8FTHG2\DATAENGINEER;" # Doit être le même que 'server'
        "DATABASE=Projet_Market_Staging;"
        "Trusted_Connection=yes;" # La ligne clé pour l'authentification Windows
    )
}

# --- 2. Configuration du Scraper ---
URL_DECOUVERTE = "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque" 
BASE_URL = "https://www.vandenborre.be"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
}


try:
    # Etape E (Extraction)
    print(f"Scraping : {URL_DECOUVERTE}...")
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(URL_DECOUVERTE, timeout=15)
    response.raise_for_status()
    print("   -> ✅ Page 'Découverte' scrapée avec succès.")

    soup = BeautifulSoup(response.content, 'html.parser')

    # Etape T (Transformation)
    product_containers = soup.find_all('div', {'class': 'js-product-container'})
    
    if not product_containers:
        print("--- Fin : Aucun conteneur produit trouvé. ---")
        exit()
        
    print(f"   -> ✅ {len(product_containers)} produits trouvés sur la page.")

    # Etape L (Load)
    print("Connexion à la base de données Staging (Auth Windows)...")
    
    # Correction de la chaîne de connexion pour l'adapter à tes infos
    conn_str = DB_CONFIG['connection_string'].replace('NOM_DE_TON_SERVEUR_SQL', DB_CONFIG['server'])
                                                
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    print("   -> ✅ Connecté à SQL Server.")

    insert_count = 0
    for container in product_containers:
        sku = container.get('data-productid')
        if not sku:
            continue

        link_tag = container.find('a', {'class': 'js-product-click'})
        if not link_tag or not link_tag.get('href'):
            continue
            
        # Reconstruction de l'URL
        relative_url = link_tag['href']
        if relative_url.startswith('//'):
            product_url = f"https:{relative_url}"
        else:
            product_url = f"{BASE_URL}{relative_url}"

        # On vérifie si ce produit est déjà en attente de scraping
        cursor.execute("SELECT 1 FROM Staging_Scraping_Queue WHERE ProductID_SKU = ? AND Status = 'pending'", (sku))
        if cursor.fetchone() is None:
            # Nouveau produit à scraper : on l'ajoute à la file d'attente
            cursor.execute(
                "INSERT INTO Staging_Scraping_Queue (ProductID_SKU, ProductURL, Status, DiscoveredAt) VALUES (?, ?, 'pending', GETDATE())",
                (sku, product_url)
            )
            insert_count += 1
    
    conn.commit()
    print(f"   -> ✅ {insert_count} nouveaux produits insérés dans Staging_Scraping_Queue.")
    
except requests.exceptions.HTTPError as err:
    print(f"\n--- ❌ ERREUR HTTP : {err} ---")
except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"\n--- ❌ ERREUR SQL Server : {sqlstate} ---")
    print("Vérifie tes 'DB_CONFIG': nom du serveur, nom de la base, et que le driver ODBC est installé.")
    print(f"Chaîne de connexion tentée : {conn_str}")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
finally:
    if 'cursor' in locals() and cursor:
        cursor.close()
    if 'conn' in locals() and conn:
        conn.close()
        print("Connexion SQL Server fermée.")

In [ ]:
def discover_all_categories(main_hub_url):
    """
    Scrappe une page "hub" principale (comme /tv-audio) pour trouver
    toutes les catégories de niveau 2 (Télévision, Home cinéma, etc.).
    
    Args:
        main_hub_url (str): L'URL de la page "hub" principale.
                            (ex: "https://www.vandenborre.be/fr/tv-audio")
        
    Retourne:
        Une liste de dictionnaires (ta "variable" de catégories). Ex:
        [
            {'category_name': 'Télévision', 'url': 'https://.../tv-audio/television'},
            {'category_name': 'Home cinéma', 'url': 'https://.../tv-audio/home-cinema'},
            ...
        ]
    """
    BASE_URL = "https://www.vandenborre.be"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    
    categories_found = []
    
    print(f"--- Lancement de la Découverte des Catégories (Étape 0a) ---")
    print(f"Scraping de la page Hub : {main_hub_url}...")
    
    try:
        session = requests.Session()
        session.headers.update(headers)
        response = session.get(main_hub_url, timeout=15)
        response.raise_for_status() # Stoppe si erreur 403, 404, etc.
        
        soup = BeautifulSoup(response.content, 'html.parser')

        # --- Extraction HTML ---
        # Le conteneur principal pour les catégories sur cette page
        main_container = soup.find('div', {'class': 'rubric-families'})
        if not main_container:
            print("❌ ERREUR : Impossible de trouver le conteneur 'div.rubric-families'.")
            return []

        # Trouver tous les blocs de catégorie
        category_blocks = main_container.find_all('div', {'class': 'accordion-image-grid'})
        
        print(f"   -> {len(category_blocks)} blocs de catégories trouvés.")

        for block in category_blocks:
            # Le lien est dans un <h2> pour les écrans larges
            link_tag = block.find('h2', {'class': 'hidden-xs'}).find('a')
            
            if not link_tag:
                continue

            category_name = link_tag.get('title')
            relative_url = link_tag.get('href')

            if category_name and relative_url:
                # Reconstruire l'URL complète
                product_url = ""
                if relative_url.startswith('//'):
                    product_url = f"https:{relative_url}"
                elif relative_url.startswith('/'):
                    product_url = f"{BASE_URL}{relative_url}"
                else:
                    product_url = relative_url
                    
                categories_found.append({
                    "category_name": category_name,
                    "url": product_url
                })
        
        print(f"--- Découverte des catégories terminée. {len(categories_found)} catégories extraites. ---")
        return categories_found

    except requests.exceptions.HTTPError as err:
        print(f"\n--- ❌ ERREUR HTTP : {err} ---")
        return []
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
        return []

In [4]:
# --- 1. Configuration SQL Server (Authentification Windows) ---
# (Basée sur votre configuration)
DB_CONFIG = {
    'server': r'LAPTOP-VT8FTHG2\DATAENGINEER', 
    'database': 'Projet_Market_Staging',
    'driver': '{ODBC Driver 17 for SQL Server}' 
}

# Construit la chaîne de connexion
conn_str = (
    f"DRIVER={DB_CONFIG['driver']};"
    f"SERVER={DB_CONFIG['server']};"
    f"DATABASE={DB_CONFIG['database']};"
    "Trusted_Connection=yes;" # Authentification Windows
)

def save_categories_to_staging(categories_to_save, univers_name):
    """
    Se connecte à la BDD et enregistre la liste des catégories 
    dans la table Staging_Category_Queue.
    
    Args:
        categories_to_save (list): La liste de dicts (le résultat de 
                                   discover_all_categories).
        univers_name (str): Le nom de la catégorie parente (ex: "TV et Audio").
    """
    print(f"\n--- Lancement du Chargement des Catégories (Étape L) ---")
    print(f"Connexion à SQL Server : {DB_CONFIG['server']}...")
    conn = None
    cursor = None
    
    try:
        conn = pyodbc.connect(conn_str, autocommit=False) # autocommit=False pour gérer la transaction
        cursor = conn.cursor()
        print("   -> ✅ Connecté à SQL Server avec succès.")

        insert_count = 0
        update_count = 0
        
        for cat in categories_to_save:
            category_name = cat['category_name']
            category_url = cat['url']
            
            # Vérifie si la catégorie existe déjà (basé sur l'URL)
            cursor.execute("SELECT CategoryQueueID FROM Staging_Category_Queue WHERE CategoryURL = ?", (category_url))
            existing_task = cursor.fetchone()
            
            if existing_task is None:
                # Cas 1: Nouvelle catégorie. On l'ajoute.
                cursor.execute(
                    """
                    INSERT INTO Staging_Category_Queue 
                        (CategoryName, CategoryURL, ParentCategoryName, Status, DiscoveredAt) 
                    VALUES (?, ?, ?, 'pending', GETDATE())
                    """,
                    (category_name, category_url, univers_name)
                )
                insert_count += 1
            else:
                # Cas 2: Catégorie déjà vue. On la réactive (met à jour le nom et le statut)
                cursor.execute(
                    """
                    UPDATE Staging_Category_Queue 
                    SET Status = 'pending', LastAttempt = NULL, CategoryName = ?, ParentCategoryName = ?
                    WHERE CategoryQueueID = ?
                    """,
                    (category_name, univers_name, existing_task.CategoryQueueID)
                )
                update_count += 1
        
        conn.commit() # Valide toutes les insertions et mises à jour
        print(f"   -> ✅ Terminé : {insert_count} nouvelles catégories insérées.")
        print(f"   -> ✅           {update_count} catégories existantes réactivées.")
        
    except pyodbc.Error as ex:
        sqlstate = ex.args[0]
        print(f"\n--- ❌ ERREUR SQL Server : {sqlstate} ---")
        print("   Vérifie tes 'DB_CONFIG': nom du serveur, nom de la base, et que le driver ODBC est installé.")
        print(f"   Chaîne de connexion tentée : {conn_str}")
        if 'conn' in locals() and conn: conn.rollback() # Annule la transaction en cas d'erreur
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
        if 'conn' in locals() and conn: conn.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
            print("Connexion SQL Server fermée.")

In [5]:
# --- Exécution du Pipeline Etape 0a ---

URL_UNIVERS = "https://www.vandenborre.be/fr/tv-audio"
NOM_UNIVERS = "TV et Audio"

# 1. Etape E (Extract)
# (Assure-toi d'avoir exécuté la Cellule 2 pour définir la fonction)
categories_data = discover_all_categories(URL_UNIVERS)

print(categories_data)

--- Lancement de la Découverte des Catégories (Étape 0a) ---
Scraping de la page Hub : https://www.vandenborre.be/fr/tv-audio...
   -> 8 blocs de catégories trouvés.
--- Découverte des catégories terminée. 8 catégories extraites. ---
[{'category_name': 'Télévision', 'url': 'https://www.vandenborre.be/fr/tv-audio/television'}, {'category_name': 'Projecteur', 'url': 'https://www.vandenborre.be/fr/tv-audio/projecteur-ecran'}, {'category_name': 'Streaming et DVD', 'url': 'https://www.vandenborre.be/fr/tv-audio/blu-ray-dvd-streaming'}, {'category_name': 'Enceinte sans fil', 'url': 'https://www.vandenborre.be/fr/tv-audio/mini-chaine-enceinte-sans-fil'}, {'category_name': 'Home cinéma', 'url': 'https://www.vandenborre.be/fr/tv-audio/home-cinema'}, {'category_name': 'Radio et Hi-Fi', 'url': 'https://www.vandenborre.be/fr/tv-audio/radio-cd-reveil'}, {'category_name': 'Casque et Écouteurs', 'url': 'https://www.vandenborre.be/fr/tv-audio/mp3-casque-ecouteurs'}, {'category_name': 'Accessoire TV et

In [6]:
# 2. Etape L (Load)
if categories_data:
    # (Assure-toi d'avoir exécuté la Cellule 3 pour définir la fonction)
    save_categories_to_staging(categories_data, univers_name=NOM_UNIVERS)
else:
    print("\n--- ÉCHEC : Aucune catégorie n'a été scrapée, rien à insérer. ---")


--- Lancement du Chargement des Catégories (Étape L) ---
Connexion à SQL Server : LAPTOP-VT8FTHG2\DATAENGINEER...
   -> ✅ Connecté à SQL Server avec succès.
   -> ✅ Terminé : 8 nouvelles catégories insérées.
   -> ✅           0 catégories existantes réactivées.
Connexion SQL Server fermée.


In [36]:
# --- Cellule 2 (CORRIGÉE) ---

def discover_all_subcategories(parent_category_name, hub_url):
    """
    Scrappe une page "hub" de catégorie (ex: /mp3-casque-ecouteurs)
    pour trouver UNIQUEMENT les sous-catégories "families"
    et ignorer les "accessoires" partagés.
    
    Args:
        parent_category_name (str): Le nom de la catégorie parente (ex: "Télévision").
        hub_url (str): L'URL de la page "hub" à scraper.
        
    Retourne:
        Une liste de dictionnaires (ta "variable" de sous-catégories).
    """
    BASE_URL = "https://www.vandenborre.be"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }

    print(f"--- Lancement Découverte Sous-Catégories (Étape 0b) ---")
    print(f"Scraping de : {hub_url} (pour parent: {parent_category_name})")
    
    subcategories_found = []
    
    try:
        session = requests.Session()
        session.headers.update(headers)
        response = session.get(hub_url, timeout=15)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')

        # --- CORRECTION DU BUG ICI ---
        # Cible UNIQUEMENT la section "families"
        main_container = soup.find('div', {'class': 'rubric-families'})

        if not main_container:
            print(f"   -> ⚠️ Aucun conteneur 'rubric-families' trouvé sur {hub_url}.")
            return []

        # Regex pour extraire "Nom (123)"
        name_count_regex = re.compile(r"(.*)\((\d+)\)")
        
        category_blocks = main_container.find_all('div', {'class': 'accordion-image-grid'})
        
        for block in category_blocks:
            link_tag = block.find('a', {'class': 'rubric-link'})
            if not link_tag: continue

            relative_url = link_tag.get('href')
            title_tag = link_tag.find(['h2', 'h3'])
            if not title_tag or not relative_url: continue

            full_text = title_tag.text.strip().replace('\xa0', ' ')
            
            url = ""
            if relative_url.startswith('//'): url = f"https:{relative_url}"
            elif relative_url.startswith('/'): url = f"{BASE_URL}{relative_url}"
            else: url = relative_url
                
            category_name = "N/A"
            item_count = 0
            
            match = name_count_regex.search(full_text)
            if match:
                category_name = match.group(1).strip()
                item_count = int(match.group(2))
            else:
                category_name = full_text
            
            if category_name != "N/A":
                subcategories_found.append({
                    "parent_category": parent_category_name,
                    "category_name": category_name,
                    "item_count": item_count,
                    "url": url
                })

        print(f"   -> {len(subcategories_found)} sous-catégories trouvées.")
        return subcategories_found

    except requests.exceptions.HTTPError as err:
        print(f"\n--- ❌ ERREUR HTTP : {err} ---")
        return []
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE : {e} ---")
        return []

In [37]:
# --- Exécution du test de l'Étape 0b ---

# On simule qu'on a lu la catégorie "Casque et Écouteurs" dans la BDD
TEST_URL = "https://www.vandenborre.be/fr/tv-audio/mp3-casque-ecouteurs"
TEST_PARENT_NAME = "Casque et Écouteurs"

# Appelle la fonction de la Cellule 2
subcategories_data = discover_all_subcategories(parent_category_name=TEST_PARENT_NAME, hub_url=TEST_URL)

if subcategories_data:
    print(f"\n--- ✅ SUCCÈS : {len(subcategories_data)} sous-catégories au total ont été découvertes ---")
    print("\nVoici la variable 'subcategories_data' :")
    
    # Affiche le JSON complet pour que tu puisses l'analyser
    print(json.dumps(subcategories_data, indent=2, ensure_ascii=False))
else:
    print("\n--- ❌ ÉCHEC : Le scraping n'a retourné aucune sous-catégorie. ---")

# Maintenant, tu peux inspecter cette sortie et décider si tu veux
# filtrer des catégories (ex: "Autres accessoires audio").

--- Lancement Découverte Sous-Catégories (Étape 0b) ---
Scraping de : https://www.vandenborre.be/fr/tv-audio/mp3-casque-ecouteurs (pour parent: Casque et Écouteurs)
   -> 11 sous-catégories trouvées.

--- ✅ SUCCÈS : 11 sous-catégories au total ont été découvertes ---

Voici la variable 'subcategories_data' :
[
  {
    "parent_category": "Casque et Écouteurs",
    "category_name": "Tous les casques",
    "item_count": 229,
    "url": "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/tous-les-casques"
  },
  {
    "parent_category": "Casque et Écouteurs",
    "category_name": "Casques audio",
    "item_count": 174,
    "url": "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/casque"
  },
  {
    "parent_category": "Casque et Écouteurs",
    "category_name": "Écouteurs",
    "item_count": 284,
    "url": "https://www.vandenborre.be/fr/mp3-casque-ecouteurs/ecouteurs"
  },
  {
    "parent_category": "Casque et Écouteurs",
    "category_name": "Casques TV",
    "item_count": 10,
    "url

In [42]:
def save_subcategories_to_staging(cursor, subcategories_to_save, parent_category_id):
    """
    Enregistre la liste des sous-catégories dans la table Staging_SubCategory_Queue.
    
    Args:
        cursor (pyodbc.Cursor): Le curseur de BDD pour exécuter les requêtes.
        subcategories_to_save (list): La liste de dicts (résultat de discover_all_subcategories).
        parent_category_id (int): L'ID (de Staging_Category_Queue) de la catégorie parente.
    """
    print(f"--- Lancement du Chargement des Sous-Catégories (Étape 0b - Load) ---")
    insert_count = 0
    update_count = 0
    ignored_count = 0

    for sub_cat in subcategories_to_save:
        cursor.execute("SELECT SubCategoryQueueID, ParentCategoryQueueID FROM Staging_SubCategory_Queue WHERE SubCategoryURL = ?", (sub_cat['url']))
        existing_sub_task = cursor.fetchone()
        
        if existing_sub_task is None:
            # Cas 1: NOUVELLE sous-catégorie. On l'insère.
            cursor.execute(
                """
                INSERT INTO Staging_SubCategory_Queue 
                    (ParentCategoryQueueID, SubCategoryName, SubCategoryURL, ItemCount, Status, DiscoveredAt) 
                VALUES (?, ?, ?, ?, 'pending', GETDATE())
                """,
                (
                    parent_category_id, 
                    sub_cat['category_name'], 
                    sub_cat['url'],           # <-- CORRIGÉ
                    sub_cat['item_count']     # <-- CORRIGÉ
                )
            )
            insert_count += 1
        
        elif existing_sub_task.ParentCategoryQueueID == parent_category_id:
            # Cas 2: On est le bon parent -> Mettre à jour
            cursor.execute(
                """
                UPDATE Staging_SubCategory_Queue 
                SET Status = 'pending', LastAttempt = NULL, SubCategoryName = ?, ItemCount = ?
                WHERE SubCategoryQueueID = ?
                """,
                (sub_cat['category_name'], sub_cat['item_count'], existing_sub_task.SubCategoryQueueID)
            )
            update_count += 1
        else:
            # Cas 3: La sous-catégorie existe mais appartient à un AUTRE parent. On ignore.
            ignored_count += 1
            pass
            
    print(f"   -> {insert_count} nouvelles sous-catégories insérées.")
    print(f"   -> {update_count} sous-catégories mises à jour.")
    if ignored_count > 0:
        print(f"   -> {ignored_count} sous-catégories ignorées (appartiennent à un autre parent).")

In [43]:
# --- 3. Exécution du Pipeline E-L (POUR TOUS les "thèmes" pending) ---
tasks_to_process = []
conn_init = None
cursor_init = None

try:
    # --- Étape Préliminaire : Récupérer la liste de TOUTES les tâches ---
    print("Connexion à SQL Server pour récupérer la liste des tâches...")
    conn_init = pyodbc.connect(conn_str)
    cursor_init = conn_init.cursor()
    
    cursor_init.execute("""
        SELECT CategoryQueueID, CategoryName, CategoryURL 
        FROM Staging_Category_Queue 
        WHERE Status = 'pending'
        ORDER BY CategoryQueueID
    """)
    tasks = cursor_init.fetchall()
    
    if not tasks:
        print("\n--- ✅ Fin : Aucune catégorie 'thème' en attente de scraping. ---")
    else:
        print(f"\n--- {len(tasks)} thèmes à traiter trouvés. Lancement du pipeline... ---")
        tasks_to_process = list(tasks)

    # On ferme la connexion initiale ICI, avant la boucle
    cursor_init.close()
    conn_init.close()
    print("   -> Connexion initiale fermée.")

    # --- Étape Principale : Boucle de traitement ---
    for task in tasks_to_process:
        task_id, task_name, task_url = task
        
        loop_conn = None
        loop_cursor = None
        
        try:
            print(f"\n-------------------------------------------------")
            print(f"--- Traitement du Thème {task_id} : '{task_name}' ---")
            
            # 2. Etape E (Extract)
            # (Assure-toi que la Cellule 2 avec 'discover_all_subcategories' a été exécutée)
            subcategories_data = discover_all_subcategories(parent_category_name=task_name, hub_url=task_url)

            loop_conn = pyodbc.connect(conn_str, autocommit=False)
            loop_cursor = loop_conn.cursor()

            if subcategories_data:
                # 3. Etape L (Load)
                save_subcategories_to_staging(loop_cursor, subcategories_data, parent_category_id=task_id)
                
                # 4. Mettre à jour le "thème" parent comme 'processed'
                loop_cursor.execute("UPDATE Staging_Category_Queue SET Status = 'processed', LastAttempt = GETDATE() WHERE CategoryQueueID = ?", (task_id))
                print(f"   -> ✅ Thème '{task_name}' marqué comme 'processed'.")
                
                loop_conn.commit() 
                
            else:
                # Le scraping n'a rien trouvé
                print(f"   -> ⚠️ Aucune sous-catégorie trouvée pour le thème '{task_name}'.")
                loop_cursor.execute("UPDATE Staging_Category_Queue SET Status = 'processed', LastAttempt = GETDATE() WHERE CategoryQueueID = ?", (task_id))
                loop_conn.commit()
                print(f"   -> ⚠️ Tâche {task_id} marquée 'processed' (sans enfants).")

        except Exception as e:
            print(f"\n--- ❌ ERREUR INATTENDUE (Tâche {task_id}) : {e} ---")
            if loop_conn: loop_conn.rollback()
            try:
                conn_fail = pyodbc.connect(conn_str, autocommit=True)
                cursor_fail = conn_fail.cursor()
                cursor_fail.execute("UPDATE Staging_Category_Queue SET Status = 'failed', LastAttempt = GETDATE() WHERE CategoryQueueID = ?", (task_id))
                cursor_fail.close()
                conn_fail.close()
                print(f"   -> ⚠️ Tâche {task_id} marquée 'failed'.")
            except Exception as e_fail:
                print(f"   -> ⚠️ CRITIQUE : Impossible de marquer la Tâche {task_id} comme 'failed'. {e_fail}")

        finally:
            if loop_cursor: loop_cursor.close()
            if loop_conn: loop_conn.close()
            
        print("   -> Pause de 3 secondes avant le prochain thème...")
        time.sleep(3)

    print("\n--- ✅ Pipeline d'Étape 0b terminé pour tous les thèmes. ---")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"\n--- ❌ ERREUR SQL Server (Connexion initiale) : {sqlstate} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE (Script principal) : {e} ---")
finally:
    # On n'a plus besoin du 'finally' ici car les connexions
    # sont gérées à l'intérieur du 'try'
    print("\nScript terminé.")

Connexion à SQL Server pour récupérer la liste des tâches...

--- 8 thèmes à traiter trouvés. Lancement du pipeline... ---
   -> Connexion initiale fermée.

-------------------------------------------------
--- Traitement du Thème 1 : 'Télévision' ---
--- Lancement Découverte Sous-Catégories (Étape 0b) ---
Scraping de : https://www.vandenborre.be/fr/tv-audio/television (pour parent: Télévision)
   -> 8 sous-catégories trouvées.
--- Lancement du Chargement des Sous-Catégories (Étape 0b - Load) ---
   -> 8 nouvelles sous-catégories insérées.
   -> 0 sous-catégories mises à jour.
   -> ✅ Thème 'Télévision' marqué comme 'processed'.
   -> Pause de 3 secondes avant le prochain thème...

-------------------------------------------------
--- Traitement du Thème 2 : 'Projecteur' ---
--- Lancement Découverte Sous-Catégories (Étape 0b) ---
Scraping de : https://www.vandenborre.be/fr/tv-audio/projecteur-ecran (pour parent: Projecteur)
   -> 4 sous-catégories trouvées.
--- Lancement du Chargement 

In [44]:
def discover_all_products(subcategory_name, category_url, expected_item_count):
    """
    Scrappe TOUTES les pages d'une sous-catégorie (ex: "Casques audio")
    en s'arrêtant lorsque le 'expected_item_count' est atteint.
    
    Args:
        subcategory_name (str): Le nom de la sous-catégorie (ex: "Casques audio").
        category_url (str): L'URL de la sous-catégorie à scraper.
        expected_item_count (int): Le nombre d'articles (ex: 174) à trouver.
        
    Retourne:
        Une liste de dictionnaires (la "variable" de produits).
    """
    BASE_URL = "https://www.vandenborre.be"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    
    all_products_found = []
    session = requests.Session()
    session.headers.update(headers)
    
    print(f"--- Lancement Découverte Produits (Étape 1) ---")
    print(f"Scraping de '{subcategory_name}' (Objectif: {expected_item_count} produits)")

    try:
        # --- 1. Découverte des métadonnées (Combien par page ?) ---
        response_page_1 = session.get(category_url, timeout=15)
        response_page_1.raise_for_status()
        soup_page_1 = BeautifulSoup(response_page_1.content, 'html.parser')
        
        count_per_page_select = soup_page_1.find('select', {'name': 'COUNTPERPAGE'})
        count_per_page = 24 # Valeur par défaut
        if count_per_page_select:
             count_per_page = int(count_per_page_select.find('option', {'selected': True}).get('value', 24))
        
        # S'il n'y a pas d'articles, on s'arrête
        if expected_item_count == 0:
             print(f"   -> ⚠️ {subcategory_name} n'a aucun article (ItemCount=0). Arrêt.")
             return []

        # Calculer le nombre de pages (ex: ceil(174 / 24) = 8)
        total_pages = math.ceil(expected_item_count / count_per_page)
        
        print(f"   -> {expected_item_count} produits sur {total_pages} pages ({count_per_page} par page).")

    except Exception as e:
        print(f"\n--- ❌ ERREUR lors de la découverte (Page 1) : {e} ---")
        return []

    # --- 2. Boucle de Scraping de toutes les pages ---
    try:
        for page_num in range(1, total_pages + 1):
            if len(all_products_found) >= expected_item_count:
                print(f"   -> Limite de {expected_item_count} produits atteinte. Arrêt.")
                break # Arrête la boucle FOR si on a tous les produits
            
            print(f"   -> Scraping de la Page {page_num}/{total_pages}...")
            
            if page_num == 1:
                soup = soup_page_1 # On ré-utilise la page 1 déjà chargée
            else:
                url_to_scrape = f"{category_url}?page={page_num}"
                response = session.get(url_to_scrape, timeout=15)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
            
            product_containers = soup.find_all('div', {'class': 'js-product-container'})
            
            if not product_containers:
                print(f"   -> ⚠️ Page {page_num} vide. On continue...")
                continue

            products_on_this_page = 0
            for container in product_containers:
                # LA CONDITION QUI RÈGLE LE PROBLÈME (ex: 192 vs 174)
                if len(all_products_found) >= expected_item_count:
                    break # Arrête la boucle FOR interne

                sku = container.get('data-productid')
                if not sku: continue 

                link_tag = container.find('a', {'class': 'js-product-click'})
                if not link_tag or not link_tag.get('href'): continue 

                relative_url = link_tag['href']
                url = ""
                if relative_url.startswith('//'): url = f"https:{relative_url}"
                elif relative_url.startswith('/'): url = f"{BASE_URL}{relative_url}"
                else: url = relative_url
                
                all_products_found.append({"sku": sku, "url": url})
                products_on_this_page += 1
            
            print(f"      -> {products_on_this_page} produits extraits de cette page.")
            
            if page_num < total_pages and len(all_products_found) < expected_item_count:
                 print("      -> Pause de 2 secondes...")
                 time.sleep(2) # Pause de politesse
        
        print(f"\n--- Scraping terminé. {len(all_products_found)} produits découverts. ---")
        return all_products_found

    except requests.exceptions.HTTPError as err:
        print(f"\n--- ❌ ERREUR HTTP pendant la boucle : {err} ---")
        return all_products_found 
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE pendant la boucle : {e} ---")
        return all_products_found

In [49]:
# --- 1. Configuration SQL Server (Authentification Windows) ---
DB_CONFIG = {
    'server': r'LAPTOP-VT8FTHG2\DATAENGINEER', 
    'database': 'Projet_Market_Staging',
    'driver': '{ODBC Driver 17 for SQL Server}' 
}

# Construit la chaîne de connexion
conn_str = (
    f"DRIVER={DB_CONFIG['driver']};"
    f"SERVER={DB_CONFIG['server']};"
    f"DATABASE={DB_CONFIG['database']};"
    "Trusted_Connection=yes;"
)

# --- 2. Fonction d'Extraction (Étape 1 - Extract) ---
def discover_all_products(subcategory_name, category_url, expected_item_count):
    """
    Scrappe TOUTES les pages d'une sous-catégorie (ex: "Casques audio")
    en s'arrêtant lorsque le 'expected_item_count' est atteint.
    """
    BASE_URL = "https://www.vandenborre.be"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    
    all_products_found = []
    session = requests.Session()
    session.headers.update(headers)
    
    print(f"--- Lancement Découverte Produits (Étape 1) ---")
    print(f"Scraping de '{subcategory_name}' (Objectif: {expected_item_count} produits)")

    try:
        # --- 1. Découverte des métadonnées (Combien par page ?) ---
        response_page_1 = session.get(category_url, timeout=15)
        response_page_1.raise_for_status()
        soup_page_1 = BeautifulSoup(response_page_1.content, 'html.parser')
        
        count_per_page_select = soup_page_1.find('select', {'name': 'COUNTPERPAGE'})
        count_per_page = 24 # Valeur par défaut
        if count_per_page_select:
             selected_option = count_per_page_select.find('option', {'selected': True})
             if selected_option:
                count_per_page = int(selected_option.get('value', 24))
        
        if expected_item_count == 0:
             print(f"   -> ⚠️ {subcategory_name} n'a aucun article (ItemCount=0). Arrêt.")
             return []

        total_pages = math.ceil(expected_item_count / count_per_page)
        print(f"   -> {expected_item_count} produits sur {total_pages} pages ({count_per_page} par page).")

    except Exception as e:
        print(f"\n--- ❌ ERREUR lors de la découverte (Page 1) : {e} ---")
        return []

    # --- 2. Boucle de Scraping de toutes les pages ---
    try:
        for page_num in range(1, total_pages + 1):
            if len(all_products_found) >= expected_item_count:
                print(f"   -> Limite de {expected_item_count} produits atteinte. Arrêt.")
                break 
            
            print(f"   -> Scraping de la Page {page_num}/{total_pages}...")
            
            if page_num == 1:
                soup = soup_page_1 
            else:
                url_to_scrape = f"{category_url}?page={page_num}"
                response = session.get(url_to_scrape, timeout=15)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
            
            product_containers = soup.find_all('div', {'class': 'js-product-container'})
            
            if not product_containers:
                print(f"   -> ⚠️ Page {page_num} vide. On continue...")
                continue

            products_on_this_page = 0
            for container in product_containers:
                if len(all_products_found) >= expected_item_count:
                    break 

                sku = container.get('data-productid')
                if not sku: continue 

                link_tag = container.find('a', {'class': 'js-product-click'})
                if not link_tag or not link_tag.get('href'): continue 

                relative_url = link_tag['href']
                url = ""
                if relative_url.startswith('//'): url = f"https:{relative_url}"
                elif relative_url.startswith('/'): url = f"{BASE_URL}{relative_url}"
                else: url = relative_url
                
                all_products_found.append({"sku": sku, "url": url})
                products_on_this_page += 1
            
            print(f"      -> {products_on_this_page} produits extraits de cette page.")
            
            if page_num < total_pages and len(all_products_found) < expected_item_count:
                 print("      -> Pause de 2 secondes...")
                 time.sleep(2) 
        
        print(f"\n--- Scraping terminé. {len(all_products_found)} produits découverts. ---")
        return all_products_found

    except requests.exceptions.HTTPError as err:
        print(f"\n--- ❌ ERREUR HTTP pendant la boucle : {err} ---")
        return all_products_found 
    except Exception as e:
        print(f"\n--- ❌ ERREUR INATTENDUE pendant la boucle : {e} ---")
        return all_products_found

# --- 3. Fonction de Chargement (Étape 1 - Load) ---
def save_products_to_staging(cursor, products_to_save, sub_category_id):
    """
    Enregistre la liste des produits dans la table Staging_Product_Queue.
    """
    print(f"--- Lancement du Chargement des Produits (Étape 1 - Load) ---")
    insert_count = 0
    update_count = 0

    for prod in products_to_save:
        cursor.execute("SELECT ProductQueueID FROM Staging_Product_Queue WHERE ProductID_SKU = ?", (prod['sku']))
        existing_prod_task = cursor.fetchone()
        
        if existing_prod_task is None:
            # Cas 1: NOUVEAU produit. On l'insère.
            cursor.execute(
                """
                INSERT INTO Staging_Product_Queue 
                    (SubCategoryQueueID, ProductID_SKU, ProductURL, Status, DiscoveredAt) 
                VALUES (?, ?, ?, 'pending', GETDATE())
                """,
                (sub_category_id, prod['sku'], prod['url'])
            )
            insert_count += 1
        else:
            # Cas 2: Produit déjà vu. On le réactive et on met à jour son URL/Parent.
            cursor.execute(
                """
                UPDATE Staging_Product_Queue 
                SET Status = 'pending', LastAttempt = NULL, ProductURL = ?, SubCategoryQueueID = ?
                WHERE ProductQueueID = ?
                """,
                (prod['url'], sub_category_id, existing_prod_task.ProductQueueID)
            )
            update_count += 1
            
    print(f"   -> {insert_count} nouveaux produits insérés.")
    print(f"   -> {update_count} produits existants mis à jour (réactivés).")

In [50]:
# --- 4. Exécution du Pipeline E-L (POUR TOUTES les sous-catégories pending) ---
tasks_to_process = []
conn_init = None
cursor_init = None

try:
    # --- Étape Préliminaire : Récupérer la liste de TOUTES les tâches ---
    print("Connexion à SQL Server pour récupérer la liste des tâches...")
    conn_init = pyodbc.connect(conn_str)
    cursor_init = conn_init.cursor()
    
    # Sélectionner TOUTES les sous-catégories "pending"
    cursor_init.execute("""
        SELECT SubCategoryQueueID, SubCategoryName, SubCategoryURL, ItemCount
        FROM Staging_SubCategory_Queue 
        WHERE Status = 'pending' AND ItemCount > 0
        ORDER BY DiscoveredAt
    """)
    tasks = cursor_init.fetchall()
    
    if not tasks:
        print("\n--- ✅ Fin : Aucune sous-catégorie en attente de scraping. ---")
    else:
        print(f"\n--- {len(tasks)} sous-catégories à traiter trouvées. Lancement du pipeline... ---")
        tasks_to_process = list(tasks)

    # On ferme la connexion initiale ICI, avant la boucle
    cursor_init.close()
    conn_init.close()
    print("   -> Connexion initiale fermée.")

    # --- Étape Principale : Boucle de traitement ---
    for task in tasks_to_process:
        task_id, task_name, task_url, task_item_count = task
        
        loop_conn = None
        loop_cursor = None
        
        try:
            print(f"\n-------------------------------------------------")
            print(f"--- Traitement de la Sous-Catégorie {task_id} : '{task_name}' ({task_item_count} articles) ---")
            
            # 2. Etape E (Extract)
            product_data = discover_all_products(
                subcategory_name=task_name, 
                category_url=task_url, 
                expected_item_count=task_item_count
            )

            loop_conn = pyodbc.connect(conn_str, autocommit=False)
            loop_cursor = loop_conn.cursor()

            if product_data:
                # 3. Etape L (Load)
                save_products_to_staging(loop_cursor, product_data, sub_category_id=task_id)
                
                # 4. Mettre à jour la sous-catégorie comme 'processed'
                loop_cursor.execute("UPDATE Staging_SubCategory_Queue SET Status = 'processed', LastAttempt = GETDATE() WHERE SubCategoryQueueID = ?", (task_id))
                print(f"   -> ✅ Sous-catégorie '{task_name}' marquée comme 'processed'.")
                
                loop_conn.commit() 
                
            else:
                # Le scraping n'a rien trouvé
                print(f"   -> ⚠️ Aucune produit trouvé pour '{task_name}'.")
                loop_cursor.execute("UPDATE Staging_SubCategory_Queue SET Status = 'failed', LastAttempt = GETDATE() WHERE SubCategoryQueueID = ?", (task_id))
                loop_conn.commit()
                print(f"   -> ⚠️ Tâche {task_id} marquée 'failed'.")

        except Exception as e:
            print(f"\n--- ❌ ERREUR INATTENDUE (Tâche {task_id}) : {e} ---")
            if loop_conn: loop_conn.rollback()
            try:
                conn_fail = pyodbc.connect(conn_str, autocommit=True)
                cursor_fail = conn_fail.cursor()
                cursor_fail.execute("UPDATE Staging_SubCategory_Queue SET Status = 'failed', LastAttempt = GETDATE() WHERE SubCategoryQueueID = ?", (task_id))
                cursor_fail.close()
                conn_fail.close()
                print(f"   -> ⚠️ Tâche {task_id} marquée 'failed'.")
            except Exception as e_fail:
                print(f"   -> ⚠️ CRITIQUE : Impossible de marquer la Tâche {task_id} comme 'failed'. {e_fail}")

        finally:
            if loop_cursor: loop_cursor.close()
            if loop_conn: loop_conn.close()
            
        print("   -> Pause de 3 secondes avant la prochaine sous-catégorie...")
        time.sleep(3)

    print("\n--- ✅ Pipeline d'Étape 1 (Découverte Produits) terminé. ---")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"\n--- ❌ ERREUR SQL Server (Connexion initiale) : {sqlstate} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE (Script principal) : {e} ---")
finally:
    print("\nScript terminé.")

Connexion à SQL Server pour récupérer la liste des tâches...

--- 57 sous-catégories à traiter trouvées. Lancement du pipeline... ---
   -> Connexion initiale fermée.

-------------------------------------------------
--- Traitement de la Sous-Catégorie 17 : 'TV OLED' (42 articles) ---
--- Lancement Découverte Produits (Étape 1) ---
Scraping de 'TV OLED' (Objectif: 42 produits)
   -> 42 produits sur 2 pages (24 par page).
   -> Scraping de la Page 1/2...
      -> 24 produits extraits de cette page.
      -> Pause de 2 secondes...
   -> Scraping de la Page 2/2...
      -> 18 produits extraits de cette page.

--- Scraping terminé. 42 produits découverts. ---
--- Lancement du Chargement des Produits (Étape 1 - Load) ---
   -> 0 nouveaux produits insérés.
   -> 42 produits existants mis à jour (réactivés).
   -> ✅ Sous-catégorie 'TV OLED' marquée comme 'processed'.
   -> Pause de 3 secondes avant la prochaine sous-catégorie...

-------------------------------------------------
--- Traiteme

In [51]:
def scrape_product_page(product_url):
    """
    Visite la page d'un produit spécifique et extrait son JSON-LD caché.
    C'est la version robuste qui évite l'erreur 'str' object has no attribute 'get'.
    
    Args:
        product_url (str): L'URL de la page produit à scraper.
        
    Retourne:
        dict: Le JSON-LD de type "Product" trouvé sur la page, ou None si échec.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    
    print(f"   -> Scraping de la page produit : {product_url}...")
    try:
        session = requests.Session()
        session.headers.update(headers)
        response = session.get(product_url, timeout=15)
        response.raise_for_status()

        soup = BeautifulSoup(response.content, 'html.parser')

        # --- Extraction du JSON "ld+json" (la méthode robuste) ---
        json_scripts = soup.find_all('script', {'type': 'application/ld+json'})
        
        for script in json_scripts:
            if not script.string:
                continue
            
            # Nettoyer le JSON (parfois il contient des caractères de contrôle)
            clean_json_string = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', script.string)
            
            data = json.loads(clean_json_string)
            
            if isinstance(data, list):
                for item in data:
                    if isinstance(item, dict) and item.get("@type") == "Product":
                        return item # On a trouvé le bon JSON
            elif isinstance(data, dict) and data.get("@type") == "Product":
                return data # On a trouvé le bon JSON
                
        # Si on arrive ici, on n'a pas trouvé le JSON produit
        print(f"   -> ⚠️ JSON-LD '@type':'Product' non trouvé sur la page.")
        return None

    except requests.exceptions.HTTPError as err:
        print(f"   -> ❌ ERREUR HTTP {err.response.status_code} sur {product_url}")
        return None
    except Exception as e:
        print(f"   -> ❌ ERREUR INATTENDUE (scrape_product_page) : {e}")
        return None

# --- 3. Fonction de Chargement (Étape 2 - Load) ---
def save_product_json_to_staging(cursor, sku, json_data, queue_id_ref):
    """
    Enregistre le JSON brut du produit dans la table Staging_VDB_Product_Page.
    
    Args:
        cursor (pyodbc.Cursor): Le curseur de BDD pour exécuter les requêtes.
        sku (str): Le SKU du produit (ex: "7819145").
        json_data (dict): Le dictionnaire JSON-LD extrait.
        queue_id_ref (int): L'ID de la tâche (de Staging_Product_Queue)
    """
    print(f"--- Lancement du Chargement du JSON Produit (Étape 2 - Load) ---")
    
    try:
        raw_json_string = json.dumps(json_data, ensure_ascii=False)
        
        # Insérer dans la table Staging_VDB_Product_Page
        cursor.execute(
            """
            INSERT INTO Staging_VDB_Product_Page 
                (ProductID_SKU, Raw_JSON_LD, QueueID_Ref) 
            VALUES (?, ?, ?)
            """,
            (sku, raw_json_string, queue_id_ref)
        )
        print(f"   -> ✅ JSON brut pour le SKU {sku} sauvegardé dans Staging_VDB_Product_Page.")
        
    except Exception as e:
        print(f"   -> ❌ ERREUR lors de l'insertion du JSON : {e}")
        raise # Propage l'erreur pour forcer un rollback

In [54]:
# --- 4. Exécution du Pipeline E-L (POUR TOUS les produits pending) ---
tasks_to_process = []
conn_init = None
cursor_init = None

try:
    # --- Étape Préliminaire : Récupérer la liste de TOUTES les tâches ---
    print("Connexion à SQL Server pour récupérer la liste des tâches...")
    conn_init = pyodbc.connect(conn_str)
    cursor_init = conn_init.cursor()
    
    # Sélectionner TOUS les produits "pending"
    cursor_init.execute("""
        SELECT ProductQueueID, ProductID_SKU, ProductURL
        FROM Staging_Product_Queue 
        WHERE Status = 'pending'
        ORDER BY ProductQueueID
    """)
    tasks = cursor_init.fetchall()
    
    if not tasks:
        print("\n--- ✅ Fin : Aucun produit en attente de scraping. ---")
    else:
        print(f"\n--- {len(tasks)} produits à traiter trouvés. Lancement du pipeline... ---")
        tasks_to_process = list(tasks)

    cursor_init.close()
    conn_init.close()
    print("   -> Connexion initiale fermée.")

    # --- Étape Principale : Boucle de traitement ---
    total_tasks = len(tasks_to_process)
    for i, task in enumerate(tasks_to_process):
        task_id, task_sku, task_url = task
        
        loop_conn = None
        loop_cursor = None
        
        print(f"\n-------------------------------------------------")
        print(f"--- Traitement Produit {i+1}/{total_tasks} (ID: {task_id}, SKU: {task_sku}) ---")
        
        try:
            # 2. Etape E (Extract)
            product_json = scrape_product_page(task_url)

            loop_conn = pyodbc.connect(conn_str, autocommit=False)
            loop_cursor = loop_conn.cursor()

            if product_json:
                # 3. Etape L (Load)
                save_product_json_to_staging(loop_cursor, task_sku, product_json, task_id)
                
                # 4. Mettre à jour la file d'attente -> 'processed'
                loop_cursor.execute("UPDATE Staging_Product_Queue SET Status = 'processed', LastAttempt = GETDATE() WHERE ProductQueueID = ?", (task_id))
                print(f"   -> ✅ Produit '{task_sku}' marqué comme 'processed'.")
                
                loop_conn.commit() 
                
            else:
                # Le scraping a échoué
                print(f"   -> ❌ Échec du scraping pour le produit '{task_sku}'.")
                loop_cursor.execute("UPDATE Staging_Product_Queue SET Status = 'failed', LastAttempt = GETDATE() WHERE ProductQueueID = ?", (task_id))
                loop_conn.commit()
                print(f"   -> ⚠️ Tâche {task_id} marquée 'failed'.")

        except Exception as e:
            print(f"\n--- ❌ ERREUR INATTENDUE (Tâche {task_id}) : {e} ---")
            if loop_conn: loop_conn.rollback()
            try:
                conn_fail = pyodbc.connect(conn_str, autocommit=True)
                cursor_fail = conn_fail.cursor()
                cursor_fail.execute("UPDATE Staging_Product_Queue SET Status = 'failed', LastAttempt = GETDATE() WHERE ProductQueueID = ?", (task_id))
                cursor_fail.close()
                conn_fail.close()
                print(f"   -> ⚠️ Tâche {task_id} marquée 'failed'.")
            except Exception as e_fail:
                print(f"   -> ⚠️ CRITIQUE : Impossible de marquer la Tâche {task_id} comme 'failed'. {e_fail}")

        finally:
            if loop_cursor: loop_cursor.close()
            if loop_conn: loop_conn.close()
            
        # PAUSE OBLIGATOIRE !
        print("   -> Pause de 2 secondes...")
        time.sleep(2)


    print("\n--- ✅ Pipeline d'Étape 2 (Scraping Produits) terminé. ---")

except pyodbc.Error as ex:
    sqlstate = ex.args[0]
    print(f"\n--- ❌ ERREUR SQL Server (Connexion initiale) : {sqlstate} ---")
except Exception as e:
    print(f"\n--- ❌ ERREUR INATTENDUE (Script principal) : {e} ---")
finally:
    print("\nScript terminé.")

Connexion à SQL Server pour récupérer la liste des tâches...

--- 533 produits à traiter trouvés. Lancement du pipeline... ---
   -> Connexion initiale fermée.

-------------------------------------------------
--- Traitement Produit 1/533 (ID: 1241, SKU: 7819390) ---
   -> Scraping de la page produit : https://www.vandenborre.be/fr/ecouteurs/jbl-reflect-aero-blue...
--- Lancement du Chargement du JSON Produit (Étape 2 - Load) ---
   -> ✅ JSON brut pour le SKU 7819390 sauvegardé dans Staging_VDB_Product_Page.
   -> ✅ Produit '7819390' marqué comme 'processed'.
   -> Pause de 2 secondes...

-------------------------------------------------
--- Traitement Produit 2/533 (ID: 1242, SKU: 7843976) ---
   -> Scraping de la page produit : https://www.vandenborre.be/fr/ecouteurs/jbl-endurance-peak-iii-blue...
--- Lancement du Chargement du JSON Produit (Étape 2 - Load) ---
   -> ✅ JSON brut pour le SKU 7843976 sauvegardé dans Staging_VDB_Product_Page.
   -> ✅ Produit '7843976' marqué comme 'pro